A series of experiments run on Bert data *after* all the sentences have already been embedded.

In [168]:
import json
import os
import random

import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from tensorflow.estimator import BaselineClassifier
from tensorflow.python.estimator.canned.dnn import DNNClassifier
from tensorflow.python.estimator.run_config import RunConfig
from tensorflow.python.estimator.training import TrainSpec, EvalSpec, train_and_evaluate

tf.logging.set_verbosity(tf.logging.ERROR)

# Helper Functions

Get Input. Assumes an 80% training, 20% dev set.

In [118]:
def get_input_fn(data_dir, num_examples=None, num_epochs=10):
    data_files = os.listdir(data_dir)
    
    # open pre-embedded data
    feature_list = []
    label_list = []
    for data_file in data_files:
        with open(os.path.join(data_dir, data_file), 'rb') as f:
            features, labels = pickle.load(f)
            feature_list.append(features)
            label_list.append(labels)
    features = np.concatenate(feature_list)
    labels = [label for labels in label_list for label in labels]
    
    # split into train and dev set
    train_features = features[0:int(0.8*len(features))]
    train_labels = labels[0:int(0.8*len(features))]
    dev_features = features[int(0.8*len(features)):len(features)]
    dev_labels = labels[int(0.8*len(features)):len(features)]
    
    train_labels = np.array(train_labels).astype('int32')
    dev_labels = np.array(dev_labels).astype('int32')
    
    if num_examples is not None:
        train_features = train_features[0:num_examples]
        train_labels = train_labels[0:num_examples]
    
#     print('{} train data points'.format(len(train_features)))
#     print('{} dev data points'.format(len(dev_features)))
    
    train_fn = tf.estimator.inputs.numpy_input_fn(
        x={'feature': train_features},
        y=train_labels,
        num_epochs=num_epochs,
        batch_size=batch_size,
        shuffle=True
    )
    dev_fn = tf.estimator.inputs.numpy_input_fn(
        x={'feature': dev_features},
        y=dev_labels,
        num_epochs=1,
        batch_size=batch_size,
        shuffle=False
    )
    return (train_fn, dev_fn)

Train and evaluate a DNN

In [154]:
def train_and_evaluate_DNN(bert_model, hidden_units, learning_rate, dropout_rate, 
                           train_num_epochs, input_size, activation_function, 
                           optimizer, output_type):
    classifier = 'DNN'
    config = tf.ConfigProto()
    run_config = RunConfig(model_dir='/home/eugenet/final_project/trained_models/imdb_{}_{}_input{}_epochs{}_hu{}_lr{}_dropout{}_activation{}_opt{}'.format(
        bert_model, classifier, input_size, train_num_epochs, '_'.join([str(x) for x in hidden_units]), learning_rate, dropout_rate, activation_function, optimizer),
                       session_config=config,
                       save_checkpoints_steps=1000)
    embedding_size = None
    if 'large' in bert_model:
        embedding_size = 1024
    elif 'small' in bert_model:
        embedding_size = 768

    activation_fn = None
    if activation_function == 'relu':
        activation_fn = tf.nn.relu
    elif activation_function == 'softmax':
        activation_fn = tf.nn.softmax
    elif activation_function == 'tanh':
        activation_fn = tf.nn.tanh
    
    opt = None
    if optimizer == 'AdaGrad':
        opt = tf.train.AdagradOptimizer(learning_rate=learning_rate)
    elif optimizer == 'AdamW':
        opt = tf.contrib.opt.AdamWOptimizer(learning_rate=learning_rate, weight_decay=0.01)
    estimator = DNNClassifier(
        hidden_units=hidden_units,
        feature_columns=[tf.feature_column.numeric_column('feature', shape=(embedding_size,))],
        n_classes=2,
        config=run_config,
        optimizer=opt,
        dropout=dropout_rate,
        activation_fn=activation_fn)
    train_input_fn, dev_input_fn = get_input_fn('/home/eugenet/final_project/cached_data/{}/'.format(bert_model), input_size, train_num_epochs)
    estimator.train(input_fn=train_input_fn)
    if output_type == 'accuracy':
        return estimator.evaluate(dev_input_fn)
    elif output_type == 'predictions':
        return estimator.predict(dev_input_fn)


# Defaults
Defaults to use for all parameters (generally a "good" set of parameters)

In [172]:
hidden_units = [1024]
learning_rate = 0.003
bert_model = 'train_uncased_large_max200'
dropout_rate = 0.1
train_num_epochs = 30
input_size = None # all
activation_function = 'relu'
optimizer = 'AdaGrad'

# Experiments

## Effect of Number of Hidden Units
Assuming we only have one hidden layer for now ("fine-tuning"). We pick out 1024 especially because that's the number of dimensions in the embedding.

In [167]:
hidden_units_list = [[1], [2], [5], [10], [50], [100], [500], [1024], [2000], [10000]]

res = []
for hu in hidden_units_list:
    result = train_and_evaluate_DNN(bert_model, hu, learning_rate, dropout_rate, train_num_epochs, input_size, activation_function, optimizer, 'accuracy')
    df_result = pd.DataFrame.from_dict(result, orient='index', columns=['{}_Hidden_Units'.format(hu)])
    res.append(df_result)
pd.concat(res, axis=1)

,[1]_Hidden_Units,[2]_Hidden_Units,[5]_Hidden_Units,[10]_Hidden_Units,[50]_Hidden_Units,[100]_Hidden_Units,[500]_Hidden_Units,[1024]_Hidden_Units,[2000]_Hidden_Units,[10000]_Hidden_Units
accuracy,0.500000,0.866400,0.869600,0.878200,0.884000,0.886800,0.891200,0.892600,0.886000,0.895000
accuracy_baseline,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
auc,0.500000,0.926781,0.934805,0.947532,0.951053,0.951325,0.953322,0.954126,0.954662,0.956050
auc_precision_recall,0.749800,0.927192,0.941591,0.947597,0.950913,0.951102,0.952879,0.953752,0.954036,0.955395
average_loss,0.693156,0.445693,0.440960,0.295502,0.285291,0.281934,0.276378,0.273639,0.279595,0.268213
label/mean,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
loss,86.644455,55.711567,55.120014,36.937691,35.661438,35.241711,34.547279,34.204906,34.949368,33.526642
precision,0.500000,0.887807,0.841211,0.867184,0.865019,0.876265,0.881138,0.889639,0.858203,0.890471
prediction/mean,0.499981,0.382510,0.631203,0.511443,0.521093,0.505989,0.509795,0.499574,0.534866,0.499886
recall,0.999200,0.838800,0.911200,0.893200,0.910000,0.900800,0.904400,0.896400,0.924800,0.900800


## Effect of Number of Epochs

In [170]:
train_num_epochs_list = [1, 2, 5, 10, 20, 30, 40, 50, 100, 150, 200]

res = []
for tne in train_num_epochs_list:
    print('Progress {}'.format(tne))
    result = train_and_evaluate_DNN(bert_model, hidden_units, learning_rate, dropout_rate, tne, input_size, activation_function, optimizer, 'accuracy')
    df_result = pd.DataFrame.from_dict(result, orient='index', columns=['{}_Epochs'.format(tne)])
    res.append(df_result)
pd.concat(res, axis=1)

Progress 1
Progress 2
Progress 5
Progress 10
Progress 20
Progress 30
Progress 40
Progress 50
Progress 100
Progress 150
Progress 200


,[10000]_Epochs,[10000]_Epochs,[10000]_Epochs,[10000]_Epochs,[10000]_Epochs,[10000]_Epochs,[10000]_Epochs,[10000]_Epochs,[10000]_Epochs,[10000]_Epochs,[10000]_Epochs
accuracy,0.875000,0.873400,0.882000,0.888600,0.885800,0.889000,0.883400,0.896800,0.893800,0.888800,0.894400
accuracy_baseline,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
auc,0.940479,0.945295,0.949209,0.952301,0.954016,0.957141,0.956707,0.957187,0.957121,0.955594,0.954894
auc_precision_recall,0.940168,0.945182,0.949012,0.952120,0.953607,0.956190,0.955996,0.956493,0.956405,0.955305,0.955739
average_loss,0.318056,0.302333,0.287993,0.279030,0.280741,0.270329,0.280951,0.265619,0.280871,0.300794,0.326381
label/mean,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
loss,39.756992,37.791618,35.999123,34.878803,35.092659,33.791084,35.118931,33.202381,35.108852,37.599197,40.797680
precision,0.866641,0.885266,0.879269,0.886282,0.861371,0.872463,0.850713,0.899356,0.883522,0.878505,0.897581
prediction/mean,0.511703,0.478039,0.494844,0.495509,0.533123,0.519556,0.546171,0.494667,0.513091,0.513341,0.492930
recall,0.886400,0.858000,0.885600,0.891600,0.919600,0.911200,0.930000,0.893600,0.907200,0.902400,0.890400


## Effect of Activaton Function

In [173]:
activation_functions = ['relu', 'softmax', 'tanh']

res = []
for af in activation_functions:
    print('Progress {}'.format(af))
    result = train_and_evaluate_DNN(bert_model, hidden_units, learning_rate, dropout_rate, train_num_epochs, input_size, af, optimizer, 'accuracy')
    df_result = pd.DataFrame.from_dict(result, orient='index', columns=['{}'.format(af)])
    res.append(df_result)
pd.concat(res, axis=1)

Progress relu
Progress softmax
Progress tanh


,relu,softmax,tanh
accuracy,0.892600,0.872600,0.887400
accuracy_baseline,0.500000,0.500000,0.500000
auc,0.956003,0.939229,0.952604
auc_precision_recall,0.955487,0.938130,0.952418
average_loss,0.268104,0.533164,0.280663
label/mean,0.500000,0.500000,0.500000
loss,33.513008,66.645523,35.082836
precision,0.892757,0.867166,0.896764
prediction/mean,0.494607,0.501442,0.479811
recall,0.892400,0.880000,0.875600


## Longer Sentences

In [177]:
bert_models = ['train_uncased_large', 'train_uncased_large_max200', 'train_uncased_large_max300']

res = []
for bm in bert_models:
    print('Progress {}'.format(bm))
    result = train_and_evaluate_DNN(bm, hidden_units, learning_rate, dropout_rate, train_num_epochs, input_size, activation_function, optimizer, 'accuracy')
    df_result = pd.DataFrame.from_dict(result, orient='index', columns=['{}'.format(bm)])
    res.append(df_result)
pd.concat(res, axis=1)

Progress train_uncased_large
Progress train_uncased_large_max200
Progress train_uncased_large_max300


,train_uncased_large,train_uncased_large_max200,train_uncased_large_max300
accuracy,0.845600,0.892400,0.895996
accuracy_baseline,0.500000,0.500000,0.500000
auc,0.929463,0.957120,0.963195
auc_precision_recall,0.927262,0.956087,0.963974
average_loss,0.344465,0.268681,0.245833
label/mean,0.500000,0.500000,0.500000
loss,43.058144,33.585110,31.466631
precision,0.867035,0.902709,0.887667
prediction/mean,0.469592,0.484267,0.509269
recall,0.816400,0.879600,0.906738


## Bert Model Type

In [175]:
bert_models = ['train_uncased_small', 'train_uncased_large', 'train_cased_small', 'train_cased_large']

res = []
for bm in bert_models:
    print('Progress {}'.format(bm))
    result = train_and_evaluate_DNN(bm, hidden_units, learning_rate, dropout_rate, train_num_epochs, input_size, activation_function, optimizer, 'accuracy')
    df_result = pd.DataFrame.from_dict(result, orient='index', columns=['{}'.format(bm)])
    res.append(df_result)
pd.concat(res, axis=1)

Progress train_uncased_small
Progress train_uncased_large
Progress train_cased_small
Progress train_cased_large


,train_uncased_small,train_uncased_large,train_cased_small,train_cased_large
accuracy,0.819800,0.851200,0.809400,0.834200
accuracy_baseline,0.500000,0.500000,0.500000,0.500000
auc,0.906073,0.928734,0.892808,0.917374
auc_precision_recall,0.905724,0.926632,0.891176,0.917563
average_loss,0.387226,0.340968,0.411950,0.364618
label/mean,0.500000,0.500000,0.500000,0.500000
loss,48.403313,42.621048,51.493755,45.577282
precision,0.816133,0.838473,0.817659,0.826495
prediction/mean,0.506833,0.514889,0.489571,0.508394
recall,0.825600,0.870000,0.796400,0.846000


## Learning Rate

In [176]:
learning_rates = [0.3, 0.03, 0.003, 0.0003, 0.00003]

res = []
for lr in learning_rates:
    print('Progress {}'.format(lr))
    result = train_and_evaluate_DNN(bert_model, hidden_units, lr, dropout_rate, train_num_epochs, input_size, activation_function, optimizer, 'accuracy')
    df_result = pd.DataFrame.from_dict(result, orient='index', columns=['{}_Learning_Rate'.format(lr)])
    res.append(df_result)
pd.concat(res, axis=1)

Progress 0.3
Progress 0.03
Progress 0.003
Progress 0.0003
Progress 3e-05


,0.3_Learning_Rate,0.03_Learning_Rate,0.003_Learning_Rate,0.0003_Learning_Rate,3e-05_Learning_Rate
accuracy,0.886800,0.888200,0.892000,0.876000,0.838400
accuracy_baseline,0.500000,0.500000,0.500000,0.500000,0.500000
auc,0.952882,0.954347,0.957370,0.945922,0.917904
auc_precision_recall,0.951977,0.954121,0.956430,0.945547,0.916720
average_loss,0.279179,0.273848,0.265801,0.300285,0.479364
label/mean,0.500000,0.500000,0.500000,0.500000,0.500000
loss,34.897385,34.230988,33.225075,37.535595,59.920483
precision,0.883122,0.886192,0.884013,0.864341,0.842788
prediction/mean,0.505200,0.493668,0.506328,0.508961,0.498696
recall,0.891600,0.890800,0.902400,0.892000,0.832000


## Dropout Rates

In [178]:
dropout_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
res = []
for dr in dropout_rates:
    print('Progress {}'.format(dr))
    result = train_and_evaluate_DNN(bert_model, hidden_units, dr, dropout_rate, train_num_epochs, input_size, activation_function, optimizer, 'accuracy')
    df_result = pd.DataFrame.from_dict(result, orient='index', columns=['{}_Dropout_Rate'.format(dr)])
    res.append(df_result)
pd.concat(res, axis=1)

Progress 0.1
Progress 0.2
Progress 0.3
Progress 0.4
Progress 0.5
Progress 0.6
Progress 0.7


,0.1_Learning_Rate,0.2_Learning_Rate,0.3_Learning_Rate,0.4_Learning_Rate,0.5_Learning_Rate,0.6_Learning_Rate,0.7_Learning_Rate
accuracy,0.886800,0.887600,0.881000,0.888400,0.881600,0.885200,0.886000
accuracy_baseline,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
auc,0.953693,0.953537,0.951620,0.954516,0.951750,0.952222,0.953714
auc_precision_recall,0.953191,0.952816,0.951542,0.954186,0.951530,0.952976,0.953920
average_loss,0.282856,0.280517,0.294300,0.277291,0.286587,0.285201,0.281847
label/mean,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
loss,35.357048,35.064629,36.787552,34.661400,35.823338,35.650112,35.230846
precision,0.869648,0.874131,0.918314,0.882585,0.887175,0.900916,0.867199
prediction/mean,0.515409,0.523047,0.462405,0.504710,0.484720,0.489444,0.525452
recall,0.910000,0.905600,0.836400,0.896000,0.874400,0.865600,0.911600


# Front and Back

In [179]:
bert_models = ['train_uncased_large_max200', 'train_uncased_large_max200_frontback']

res = []
for bm in bert_models:
    print('Progress {}'.format(bm))
    result = train_and_evaluate_DNN(bm, hidden_units, learning_rate, dropout_rate, train_num_epochs, input_size, activation_function, optimizer, 'accuracy')
    df_result = pd.DataFrame.from_dict(result, orient='index', columns=['{}'.format(bm)])
    res.append(df_result)
pd.concat(res, axis=1)

Progress train_uncased_large_max200
Progress train_uncased_large_max200_frontback


,train_uncased_large_max200,train_uncased_large_max200_frontback
accuracy,0.892800,0.816400
accuracy_baseline,0.500000,0.500000
auc,0.957152,0.909455
auc_precision_recall,0.956171,0.912010
average_loss,0.268967,0.376141
label/mean,0.500000,0.500000
loss,33.620853,47.017639
precision,0.893115,0.814388
prediction/mean,0.497026,0.495450
recall,0.892400,0.819600


# Optimizers

In [180]:
optimizers = ['AdaGrad', 'AdamW']

res = []
for opt in optimizers:
    print('Progress {}'.format(opt))
    result = train_and_evaluate_DNN(bert_model, hidden_units, learning_rate, dropout_rate, train_num_epochs, input_size, activation_function, opt, 'accuracy')
    df_result = pd.DataFrame.from_dict(result, orient='index', columns=['{}'.format(opt)])
    res.append(df_result)
pd.concat(res, axis=1)

Progress AdaGrad
Progress AdamW


,AdaGrad,AdamW
accuracy,0.893400,0.873200
accuracy_baseline,0.500000,0.500000
auc,0.956967,0.946567
auc_precision_recall,0.955776,0.946463
average_loss,0.270448,0.313827
label/mean,0.500000,0.500000
loss,33.805943,39.228344
precision,0.892615,0.835853
prediction/mean,0.498137,0.553442
recall,0.894400,0.928800
